# Natural Language Processing + Decision Science

👩🏻‍🏫 In this challenge, we will combine:
* 🗣 Natural Language Processing 
* 📊 Decision Science 

🎯 The goal is to understand the bad reviews of the products and the sellers on Olist

In [ ]:
# Data Manipulation
import numpy as np
import pandas as pd
pd.set_option("max_columns",None)

# Olist packages
from olist.data import Olist
from olist.review import Review
from olist.order import Order
from olist.product_updated import Product
from olist.seller_updated import Seller

# Machine Learning
from sklearn.pipeline import make_pipeline

# Language Processing
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
import string
import unidecode

# Vectorizers and NLP Models
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

🕵🏻‍♂️ Imagine that the CEO of Olist [Tiago Dalvi](https://www.linkedin.com/in/tiagodalvi/) ask you to read and understand the reviews.

- What did customers say about their order if they rated it 1? 2? 3?
- What are the most frequent bad reviews about...
    - the worst rated products ?
    - the wort sellers ?

## (0) Setup

👩🏻‍🏫 If you followed the `Decision Science` module, you already have the `olist` package installed and importable. *You can skip this section* and move to the **Data collection** section.

### (0.1) Import `olist` package`

Download a fresh version of the `olist` package:

```bash
mkdir ~/code/lewagon
cd ~/code/lewagon
git clone git@github.com:lewagon/olist.git
cd olist
git fetch
git checkout full-package
```

### (0.2) Download the datasets

- Download the datasets from Kaggle https://www.kaggle.com/olistbr/brazilian-ecommerce
- Unzip them into the `/data/csv` directory of the `olist` package:

```bash
.
├── README.md
├── data
│   └── csv
│       ├── olist_customers_dataset.csv
│       ├── olist_geolocation_dataset.csv
│       ├── olist_order_items_dataset.csv
│       ├── olist_order_payments_dataset.csv
│       ├── olist_order_reviews_dataset.csv
│       ├── olist_orders_dataset.csv
│       ├── olist_products_dataset.csv
│       ├── olist_sellers_dataset.csv
│       └── product_category_name_translation.csv
├── notebooks
├── olist
│   ├── README.md
│   ├── __init__.py
│   ├── data.py
│   ├── order.py
│   ├── product.py
│   ├── product_updated.py
│   ├── review.py
│   ├── seller.py
│   ├── seller_updated.py
│   └── utils.py
├── requirements.txt
└── setup.p
```

### (0.3) Install the `olist` package

```bash
pip install -e .
```

⚠️ Restart the kernel.

## (1) Data Collection and Preparation

👉 The way we designed the `reviews` DataFrame will help us focus on the `product categories`.

In [ ]:
reviews = Review().get_training_data()
reviews.head()

,review_id,length_review,review_score,order_id,product_category_name
0,7bc2406110b926393aa56f80a40eba40,0,4,73fc7af87114b39712e6da79b0a377eb,esporte_lazer
1,80e641a11e56f04c1ad469d5645fdfde,0,5,a548910a1c6147796b98fdf73dbeba33,informatica_acessorios
2,228ce5500dc1d8e020d8d1322874b6f0,0,5,f9e4b658b201a9f2ecdecbb34bed034b,informatica_acessorios
3,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim
4,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer


👉 We can retrieve the reviews from `order_reviews` within the Olist class.

In [ ]:
data = Olist().get_data()

In [ ]:
reviews_data = data['order_reviews']
reviews_data.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


🪤 We need to collect some information about the orders because:
- Back then, customers could review an order even before receiving it
- They could also rate an order that they hadn't received

In [ ]:
orders = data['orders']
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


💥 Let's merge these three datasets and apply some operations

In [ ]:
df = reviews.merge(reviews_data).merge(orders)
df.head()

,review_id,length_review,review_score,order_id,product_category_name,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,7bc2406110b926393aa56f80a40eba40,0,4,73fc7af87114b39712e6da79b0a377eb,esporte_lazer,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59,41dcb106f807e993532d446263290104,delivered,2018-01-11 15:30:49,2018-01-11 15:47:59,2018-01-12 21:57:22,2018-01-17 18:42:41,2018-02-02 00:00:00
1,80e641a11e56f04c1ad469d5645fdfde,0,5,a548910a1c6147796b98fdf73dbeba33,informatica_acessorios,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13,8a2e7ef9053dea531e4dc76bd6d853e6,delivered,2018-02-28 12:25:19,2018-02-28 12:48:39,2018-03-02 19:08:15,2018-03-09 23:17:20,2018-03-14 00:00:00
2,228ce5500dc1d8e020d8d1322874b6f0,0,5,f9e4b658b201a9f2ecdecbb34bed034b,informatica_acessorios,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24,e226dfed6544df5b7b87a48208690feb,delivered,2018-02-03 09:56:22,2018-02-03 10:33:41,2018-02-06 16:18:28,2018-02-16 17:28:48,2018-03-09 00:00:00
3,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06,de6dff97e5f1ba84a3cd9a3bc97df5f6,delivered,2017-04-09 17:41:13,2017-04-09 17:55:19,2017-04-10 14:24:47,2017-04-20 09:08:35,2017-05-10 00:00:00
4,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53,5986b333ca0d44534a156a52a8e33a83,delivered,2018-02-10 10:59:03,2018-02-10 15:48:21,2018-02-15 19:36:14,2018-02-28 16:33:35,2018-03-09 00:00:00


🚓 Remove the reviews that were written even before receiving the order.

In [ ]:
rated_after_received = df["order_delivered_customer_date"] < df["review_creation_date"]
df = df[rated_after_received]

🚓 Remove the reviews for undelivered orders.

In [ ]:
delivered = df['order_status'] == "delivered"
df = df[delivered]

✍️ As some people fill in only the title or the body of a review, it may be a good idea to aggregate them together.



<details>
    <summary>💡<i>Hint</i></summary>

* Drop rows with either a missing title or a missing comment
* Refer to the documentation [pandas.DataFrame.dropna()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html) and have a look at the `how` argument...

In [ ]:
df = df.dropna(subset=['review_comment_title','review_comment_message'],
              how = "all")

df['full_review'] = df["review_comment_title"].fillna('') + " " \
            + df['review_comment_message'].fillna('')

🕵🏻 Let's focus on some columns of interest:

In [ ]:
# Focus on certain columns
columns_of_interest = ['review_id',
                       'length_review',
                       'review_score',
                       'order_id',
                       'product_category_name',
                       'full_review']
df = df[columns_of_interest]

In [ ]:
df.head()

,review_id,length_review,review_score,order_id,product_category_name,full_review
3,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,Recebi bem antes do prazo estipulado.
4,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,Parabéns lojas lannister adorei comprar pela ...
9,8670d52e15e00043ae7de4c01cc2fe06,174,4,b9bf720beb4ab3728760088589c62129,eletroportateis,recomendo aparelho eficiente. no site a marca ...
12,4b49719c8a200003f700d3d986ea1a19,45,4,9d6f15f95d01e79bd1349cc208361f09,beleza_saude,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
15,3948b09f7c818e2d86c9a546758b2335,56,5,e51478e7e277a83743b6f9991dbfa3fb,informatica_acessorios,"Super recomendo Vendedor confiável, produto ok..."


## (2) Text Cleaning

🧹 Create a function `cleaning(sentence)` and apply it to the reviews.

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
import string
import unidecode


def cleaning(sentence):
    
    # Basic cleaning
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercasing 
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## removing numbers
    sentence = unidecode.unidecode(sentence) # remove accents
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## removing punctuation
    # Advanced cleaning        
    tokenized_sentence = word_tokenize(sentence) ## tokenizing 
    stop_words = set(stopwords.words('portuguese')) ## defining stopwords
    tokenized_sentence = [w for w in tokenized_sentence 
                                  if not w in stop_words] ## remove stopwords
    lemmatized_sentence = [WordNetLemmatizer().lemmatize(word, pos = "v")  # v --> verbs
              for word in tokenized_sentence]
    cleaned_sentence = ' '.join(word for word in lemmatized_sentence)
    return cleaned_sentence

In [ ]:
df["full_review_cleaned"] = df["full_review"].apply(cleaning)
df["full_review_cleaned"]

3                         recebi bem ante prazo estipulado
4        parabens lojas lannister adorei comprar intern...
9        recomendo aparelho eficiente site marca aparel...
12                         pouco travandopelo valor ta boa
15       super recomendo vendedor confiavel produto ok ...
                               ...                        
98639               produto recebi acordo compra realizada
98642    entregou dentro prazo produto chegou condicoes...
98648    produto nao enviado nf nao existe venda nf cer...
98654    excelente mochila entrega super rapida super r...
98656    produto chegou ja devolver pois defeito nao se...
Name: full_review_cleaned, Length: 36148, dtype: object

## (3) Analysis of bad reviews

### (3.1) Dataset with low review scores

😱 What is the proportion of reviews with a rating between 1 and 3 ? 

In [ ]:
round(df["review_score"].value_counts(normalize = True),2)

5    0.57
4    0.16
1    0.13
3    0.09
2    0.05
Name: review_score, dtype: float64

🕵🏻‍♂️ Let's focus on these reviews...

In [ ]:
threshold = 3
bad = df["review_score"] <= threshold
df = df[bad]
df.shape

(9658, 7)

In [ ]:
df.head()

,review_id,length_review,review_score,order_id,product_category_name,full_review,full_review_cleaned
53,e233e51d11511bf30e568c76360ace52,97,1,548df2c6e5f089574614894bca78acf5,eletronicos,recebi somente 1 controle Midea Split ESTILO....,recebi somente controle midea split estilo fal...
75,8b230a1373c6dc4bd867099fda1d7039,60,3,071251fe3b3493294536f03737a8a679,ferramentas_jardim,Eu comprei duas unidades e só recebi uma e ag...,comprei duas unidades so recebi agora faco
77,cb2fc3e5711b5ae85e0491ee18af63ed,72,3,34e6d418f368f8079ae152bc178bc66a,beleza_saude,"Produto bom, porém o que veio para mim não co...",produto bom porem veio mim nao condiz foto anu...
78,60c714ed14cef913944a3147094a4742,36,1,9ac05114800f02bfaa783bd76842dbe2,moveis_decoracao,"Produto muito inferior, mal acabado.",produto inferior mal acabado
89,0bd4dcc4f6c4621baf37f73495cad8c4,16,3,a11cd01ac67beef7e8bf09740d8a35c1,esporte_lazer,Entrega no prazo,entrega prazo


### (3.2) Vectorizing

🔡 ➡️ 🔢 Vectorize your texts. 

* Make sure to take into accounts bigrams.
* Set a `max_df` to $0.75$ to remove too frequent words
* Spoiler alert: you will end up with $20k+$ words... let's keep only `max_features` = $5000$ for this challenge.

In [ ]:
vectorizer = TfidfVectorizer(max_df = 0.75, max_features = 5000, ngram_range=(1,2))
vectorized_reviews = pd.DataFrame(vectorizer.fit_transform(df["full_review_cleaned"]).toarray(),
                                 columns = vectorizer.get_feature_names_out())

print(f" vectorized_reviews.shape = {vectorized_reviews.shape}")

 vectorized_reviews.shape = (9658, 5000)


### (3.3) LDA

🕵🏻‍♂️ Fit an LDA:
- Choose `n_components = 3`
- Show the Document Mixture of Topics  with *.transform()*
- Show the Topic Mixture with *.components_*

In [ ]:
n_components = 3

lda = LatentDirichletAllocation(n_components = n_components)
lda.fit(vectorized_reviews)

LatentDirichletAllocation(n_components=3)

#### Document Mixture (of Topics)

In [ ]:
document_mixture = lda.transform(vectorized_reviews)
document_mixture.shape

(9658, 3)

In [ ]:
round(pd.DataFrame(document_mixture, 
                   columns = [f"Topic {i+1}" for i in range(n_components)])
      ,2)

,Topic 1,Topic 2,Topic 3
0,0.83,0.09,0.08
1,0.76,0.08,0.16
2,0.07,0.67,0.26
3,0.10,0.80,0.10
4,0.13,0.13,0.75
...,...,...,...
9653,0.68,0.08,0.25
9654,0.11,0.78,0.11
9655,0.46,0.06,0.48
9656,0.61,0.31,0.08


👉 Let's report the most important topic for each review

In [ ]:
df["most_important_topic"] = np.argmax(document_mixture, axis = 1)
df.head()

,review_id,length_review,review_score,order_id,product_category_name,full_review,full_review_cleaned,most_important_topic
53,e233e51d11511bf30e568c76360ace52,97,1,548df2c6e5f089574614894bca78acf5,eletronicos,recebi somente 1 controle Midea Split ESTILO....,recebi somente controle midea split estilo fal...,0
75,8b230a1373c6dc4bd867099fda1d7039,60,3,071251fe3b3493294536f03737a8a679,ferramentas_jardim,Eu comprei duas unidades e só recebi uma e ag...,comprei duas unidades so recebi agora faco,0
77,cb2fc3e5711b5ae85e0491ee18af63ed,72,3,34e6d418f368f8079ae152bc178bc66a,beleza_saude,"Produto bom, porém o que veio para mim não co...",produto bom porem veio mim nao condiz foto anu...,1
78,60c714ed14cef913944a3147094a4742,36,1,9ac05114800f02bfaa783bd76842dbe2,moveis_decoracao,"Produto muito inferior, mal acabado.",produto inferior mal acabado,1
89,0bd4dcc4f6c4621baf37f73495cad8c4,16,3,a11cd01ac67beef7e8bf09740d8a35c1,esporte_lazer,Entrega no prazo,entrega prazo,2


#### Topic Mixture (of Words)

In [ ]:
topic_mixture = pd.DataFrame(lda.components_, 
                             columns = vectorizer.get_feature_names_out())
topic_mixture.shape

(3, 5000)

In [ ]:
topic_mixture

abaixo    aberta    aberto      abri  abri caixa  abri embalagem  \
0  0.347893  0.598400  0.355573  8.731675    0.792742        1.091973   
1  3.602708  2.470902  7.729957  2.795721    0.584628        1.310103   
2  0.397029  4.730480  0.788683  0.592689    0.912531        0.352813   

   abri reclamacao   abrindo     abrir  abrir embalagem   absurdo   acabada  \
0         4.495764  0.592542  4.641205         0.357529  6.351896  0.351331   
1         0.345856  1.284689  3.693474         2.007148  3.083325  2.042055   
2         0.348484  1.301583  4.517060         0.466159  3.257768  0.356358   

    acabado  acabamento  acabamento nao  acabamento ruim  acabamentos  \
0  0.334493    0.348581        0.397894         0.334291     0.335762   
1  5.226098   15.785174        1.020107         3.759522     1.791807   
2  0.367522    7.278171        1.108985         0.337648     0.436884   

     acabei  acabei recebendo    acabou    aceita    acende  acessorios  \
0  2.051983          2.008887  1.315780  2.601471  0.592249    3.336629   
1  1.562811          0.344398  2.252037  0.396981  1.224157    0.888049   
2  3.574593          0.379059  0.520045  0.344205  0.417400    0.510053   

    achando      achei  achei bem  achei demorou  achei material  \
0  0.497355   0.359102   0.337964       0.334864        0.335043   
1  1.724660  18.476637   1.116415       0.653522        0.807226   
2  1.807147  41.498271   1.386112       3.522273        2.668684   

   achei melhor  achei pequeno  achei pouco  achei prazo  achei produto  \
0      0.336849       0.336817     0.334491     0.338566       0.342477   
1      1.316090       0.360055     0.409530     0.335301       3.856659   
2      1.107035       3.481756     3.996977     3.289263       5.078708   

        acho  acho deveria  acho nao  acho veio     acima       aco  \
0   2.285665      1.261285  0.382648   0.375798  1.152452  0.834675   
1   4.315704      0.403840  1.557075   0.798149  1.109481  1.792652   
2  12.057032      2.307270  0.747002   1.773138  0.712952  0.337808   

   acompanha  acompanhamento  acompanhar  acontece  acontecendo  aconteceu  \
0   1.921506        0.696390    1.557975  2.064250     1.547841  12.128405   
1   0.626363        0.338710    0.469754  3.104289     0.339671   0.379188   
2   0.822780        2.171125    1.283832  0.338366     1.197852   0.365902   

   aconteceu produto  acontecido    acordo  acostumada  acostumada comprar  \
0           2.544632    2.204158  2.219123    0.334412            0.334468   
1           0.428662    0.344237  5.947171    1.932179            1.636626   
2           0.348632    1.519691  2.732559    0.383195            0.383203   

   acredito  acredito nao  acrilico  acrilico fica     acusa  adaptador  \
0  0.442670      0.339391  0.999016       0.333968  1.397988   0.368179   
1  6.030191      0.981859  0.926503       0.342086  0.699588   0.387463   
2  5.001738      1.254879  3.547909       1.828035  0.354504   3.896258   

   adequada  adequado   adesiva   adesivo  adesivo parede  adesivo pra  \
0  0.343149  1.034964  0.351830  5.083423        1.585275     1.201765   
1  0.356204  1.401563  2.546310  1.375649        0.334245     0.342707   
2  2.748400  0.338669  0.339073  1.275213        0.340587     0.334221   

   adesivos   adianta    adorei   adquiri  adquiri produto  adquirido  \
0  1.423435  0.522931  0.355854  5.959942         1.862329   1.422327   
1  1.657519  0.344976  0.359861  1.316206         0.430209   3.572614   
2  0.349990  1.780971  7.879146  0.352211         0.335519   2.285126   

   adquirido nao  adquiridos  adquirir    adulto    afetou  afetou adesivo  \
0       0.346811    1.298910  0.361015  1.509164  1.503072        1.201765   
1       2.057503    0.384421  1.222566  0.372486  0.702150        0.342707   
2       0.921266    1.213285  0.599079  0.336993  0.387843        0.334221   

     afinal   agencia  agencia correio  agencia correios  agilidade  agilizar  \
0  1.551870  0.337589         0

#### Topics

🎁 We provided you with some functions:
* `topic_word` returns the top words with their weights for one topic
* `print_topics` prints the different topics found by the LDA with their topwords

In [ ]:
def topic_word(vectorizer, model, topic, topwords, with_weights = True):
    topwords_indexes = topic.argsort()[:-topwords - 1:-1]
    if with_weights == True:
        topwords = [(vectorizer.get_feature_names_out()[i], round(topic[i],2)) for i in topwords_indexes]
    if with_weights == False:
        topwords = [vectorizer.get_feature_names_out()[i] for i in topwords_indexes]
    return topwords

In [ ]:
def print_topics(vectorizer, model, topwords):
    
    for idx, topic in enumerate(model.components_):
        print("-"*20)
        print("Topic %d:" % (idx))
        print(topic_word(vectorizer, model, topic, topwords))
        

🕵🏻‍♂️ Print the topics with their topwords:

In [ ]:
print_topics(vectorizer, lda, topwords = 5)

--------------------
Topic 0:
[('recebi', 254.73), ('nao', 216.45), ('comprei', 178.98), ('nao recebi', 142.9), ('so', 139.6)]
--------------------
Topic 1:
[('produto', 227.84), ('nao', 160.12), ('veio', 132.45), ('qualidade', 92.95), ('defeito', 83.32)]
--------------------
Topic 2:
[('bom', 229.8), ('entrega', 145.69), ('produto', 135.99), ('prazo', 128.18), ('nao', 118.6)]


🇧🇷 A bit of Brazilian Portuguese words here:
- _cadeiras = chairs_
- _producto = product_
- _bom = good_
- _comprei = bought_
- _veio = came_
- _duas = two_
- _nao = not_
- _entregue = delivered_
- _pecas = part_
- _ainda = yet_
- _recebi = received_

👉 Show the top words associated to a topic

In [ ]:
topic_word_mixture = [topic_word(vectorizer, lda, topic, topwords = 5, with_weights = False)
                      for topic in lda.components_]
topic_word_mixture

[['recebi', 'nao', 'comprei', 'nao recebi', 'so'],
 ['produto', 'nao', 'veio', 'qualidade', 'defeito'],
 ['bom', 'entrega', 'produto', 'prazo', 'nao']]

In [ ]:
df["most_important_words"] = df["most_important_topic"].apply(lambda i: topic_word_mixture[i])

In [ ]:
df[["review_id", 
        "review_score", 
        "product_category_name",
        "full_review_cleaned",
        "most_important_topic",
        "most_important_words"]
      ].head(3)

,review_id,review_score,product_category_name,full_review_cleaned,most_important_topic,most_important_words
53,e233e51d11511bf30e568c76360ace52,1,eletronicos,recebi somente controle midea split estilo fal...,0,"[recebi, nao, comprei, nao recebi, so]"
75,8b230a1373c6dc4bd867099fda1d7039,3,ferramentas_jardim,comprei duas unidades so recebi agora faco,0,"[recebi, nao, comprei, nao recebi, so]"
77,cb2fc3e5711b5ae85e0491ee18af63ed,3,beleza_saude,produto bom porem veio mim nao condiz foto anu...,1,"[produto, nao, veio, qualidade, defeito]"


## (3.4) Pipeline Tf-Idf and LDA

In [ ]:
from sklearn import set_config
set_config("diagram")

🔨 Create a Pipeline that chains your previous Vectorizer and the LDA.

Fit it on the cleaned texts.

In [ ]:
%%time
# Parameters
max_df = 0.75
max_features = 5000
ngram_range = (1,2)

# Pipeline Vectorizer + LDA
pipeline = make_pipeline(
    TfidfVectorizer(max_df = max_df,
                    max_features = max_features,
                    ngram_range = ngram_range),
    LatentDirichletAllocation(n_components = n_components)
)

# Fit the pipeline on the cleaned texts
pipeline.fit(df["full_review_cleaned"])

CPU times: user 17.1 s, sys: 87.3 ms, total: 17.1 s
Wall time: 17.3 s


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_df=0.75, max_features=5000,
                                 ngram_range=(1, 2))),
                ('latentdirichletallocation',
                 LatentDirichletAllocation(n_components=3))])

💡 If you try to access the components with `pipeline.components_`, it will NOT work because Pipeline doesn't have a `components_`. However, you can use `pipeline._final_estimator` to access the LDA. And from this, you can access the topics!

In [ ]:
pipeline._final_estimator

LatentDirichletAllocation(n_components=3)

In [ ]:
pipeline._final_estimator.components_

array([[3.59021282, 0.71236482, 1.1439959 , ..., 0.34978534, 8.07634649,
        5.95005341],
       [0.37532863, 6.74895969, 7.37959494, ..., 0.33849303, 0.39644635,
        0.88262968],
       [0.3820883 , 0.33845716, 0.35062264, ..., 2.86945187, 0.40136583,
        0.35505877]])

**Document Mixture** with the Pipeline:

In [ ]:
# Transform the original cleaned texts with the pipeline
# Indeed, there is no need to get the vectorized texts first since it's done through the Pipeline
document_mixture = pipeline.transform(df["full_review_cleaned"]) 
document_mixture.shape 

(9658, 3)

**Topic Mixture** with the Pipeline:

In [ ]:
topic_mixture = pd.DataFrame(pipeline._final_estimator.components_)
topic_mixture.shape

(3, 5000)

In [ ]:
topic_mixture

0         1         2         3         4         5         6     \
0  3.590213  0.712365  1.143996  8.379278  0.556057  1.539410  4.453618   
1  0.375329  6.748960  7.379595  3.254666  0.666179  0.869331  0.336761   
2  0.382088  0.338457  0.350623  0.486141  1.067665  0.346147  0.399725   

       7          8         9         10        11        12         13    \
0  0.845277   0.666323  0.423225  0.481754  0.338435  5.164598   1.160313   
1  1.981639  11.667744  0.471393  9.899905  1.251912  0.422040  21.828101   
2  0.351899   0.517670  1.936217  2.311330  1.159397  0.341475   0.423513   

       14        15        16        17        18        19        20    \
0  0.349368  0.337130  0.769481  1.378379  1.744956  1.879448  0.348462   
1  1.080660  3.758952  1.399492  5.060887  0.589685  0.618235  2.652644   
2  1.096958  0.335379  0.395480  0.750121  0.397702  1.590179  0.341550   

       21        22        23         24        25        26        27    \
0  0.358835  0.361442  1.488519  11.024964  1.219953  0.348464  0.335102   
1  1.519684  1.590872  2.002365  46.941567  0.797126  3.824220  3.135738   
2  0.355287  2.782416  0.538277   2.367479  0.823413  0.337974  0.340113   

       28        29        30        31        32         33        34    \
0  0.796403  0.344092  0.382988  0.342579  8.066009   0.368990  0.343419   
1  0.404422  0.377613  4.004682  3.285966  0.872449  17.930301  3.283275   
2  1.559148  3.456923  0.353329  0.334584  0.339385   0.359110  0.345701   

       35        36        37        38        39        40        41    \
0  0.335741  0.544728  0.343564  0.335788  0.341638  0.358024  0.372820   
1  2.013989  1.349239  0.346061  0.954964  2.307493  2.501521  2.400885   
2  0.336994  1.053117  2.285260  1.674383  0.721518  0.346681  0.537855   

       42        43        44        45        46        47        48    \
0  0.385744  0.349765  4.359925  2.512662  0.341172  0.663351  0.345731   
1  4.771980  0.518318  1.604635  0.443198  0.360923  6.434887  1.961658   
2  0.349182  2.217282  6.908935  0.366066  3.365991  3.800616  0.342397   

       49        50        51        52        53        54        55    \
0  0.343903  4.276802  0.386262  2.135708  1.835801  1.025700  0.395823   
1  1.672125  6.448087  1.781993  0.464102  0.334041  0.342739  2.130335   
2  0.338269  0.749711  0.407874  2.873617  0.334247  1.083641  2.125741   

       56        57        58        59        60        61        62    \
0  1.436287  0.356462  0.337761  2.635479  0.454023  0.333820  0.668084   
1  1.659422  0.339858  2.564635  2.600645  0.341373  0.333874  1.443800   
2  0.352043  2.078875  0.334816  2.498161  1.464710  1.210999  1.319058   

       63        64        65        66        67        68        69    \
0  0.350976  0.344233  6.770167  1.898584  0.613983  0.420361  0.669634   
1  0.379644  0.382133  0.453022  0.346636  2.185789  0.441945  1.061294   
2  1.918257  7.868495  0.405170  0.382836  4.480294  2.463274  1.165688   

       70        71        72        73        74        75        76    \
0  0.340895  0.971906  0.486271  0.333820  0.348169  0.340748  0.336241   
1  1.225533  0.600612  0.413135  0.333874  0.627634  2.319735  0.374398   
2  0.616232  0.646125  1.693659  1.210999  1.586376  5.890683  1.742732   

       77        78        79         80        81        82        83    \
0  0.342231  3.381688  1.493052  36.307947  0.432086  1.842078  7.555880   
1  0.378513  0.722894  0.461573   3.719854  0.402716  0.487461  0.507018   
2  4.365251  0.371383  0.642510  16.318027  1.987012  0.654087  2.443602   

        84        85        86        87        88        89        90    \
0  19.052321  0.609490  0.471669  2.166458  2.788454  0.370380  0.336513   
1   0.353141  1.207445  1.213446  0.334112  0.361258  1.470826  2.081812   
2   0.855825  1.537878  0.504274  0.355852  1.621454  0.350150  0.397892   

       91        92        93         94        95         96        9

## (4) 🎁 Product Categories

### (4.1) Groupby product categories

📈 Group the dataset by `product_category_name` and inspect their performances

In [ ]:
# Product categories by performance - look at the count, mean, median and std
product_categories = df.groupby(by = 'product_category_name').agg({
        'review_score': ["count", "mean", "median", "std"]
    })

# Removing products which were sold less than a certain times for the analysis
cutoff = 50
product_categories = product_categories[product_categories[("review_score", "count")] > cutoff]

# Sorting the product categories by performance
product_categories = product_categories.sort_values(by = [('review_score', 'mean'), 
                                                          ('review_score', 'std')], 
                                                    ascending = [False, True])
product_categories

review_score                           
                                         count      mean median       std
product_category_name                                                    
consoles_games                              92  1.956522    2.0  0.924787
fashion_bolsas_e_acessorios                155  1.948387    2.0  0.917319
malas_acessorios                            66  1.924242    2.0  0.949666
papelaria                                  155  1.922581    2.0  0.922556
casa_construcao                             66  1.909091    2.0  0.836242
utilidades_domesticas                      586  1.889078    2.0  0.888473
cool_stuff                                 298  1.885906    2.0  0.906618
relogios_presentes                         589  1.881154    2.0  0.905088
pet_shop                                   147  1.857143    2.0  0.883641
moveis_escritorio                          261  1.854406    1.0  0.920843
cama_mesa_banho                           1221  1.850942    2.0  0.890235
esporte_lazer                              626  1.846645    2.0  0.899136
casa_conforto                               57  1.842105    2.0  0.902169
beleza_saude                               716  1.840782    1.0  0.908275
ferramentas_jardim                         318  1.817610    1.0  0.904635
telefonia                                  488  1.815574    2.0  0.875636
moveis_decoracao                           743  1.802153    1.0  0.888620
brinquedos                                 298  1.788591    1.0  0.890808
bebes                                      244  1.786885    1.0  0.877046
eletronicos                                248  1.778226    1.0  0.883502
construcao_ferramentas_construcao           71  1.774648    1.0  0.881511
automotivo                                 370  1.770270    1.0  0.870275
perfumaria                                 235  1.723404    1.0  0.879543
informatica_acessorios                     797  1.708908    1.0  0.867089
eletrodomesticos                            84  1.630952    1.0  0.875087
eletroportateis                             54  1.518519    1.0  0.770708

### (4.2) Worst product categories

👎 Store the five worst categories in terms of *average review score* into a variable called `worst_products`

In [ ]:
worst_products = product_categories.tail(5).sort_values(by = [("review_score", "count")],
                                                       ascending = False)
worst_products

review_score                           
                              count      mean median       std
product_category_name                                         
informatica_acessorios          797  1.708908    1.0  0.867089
automotivo                      370  1.770270    1.0  0.870275
perfumaria                      235  1.723404    1.0  0.879543
eletrodomesticos                 84  1.630952    1.0  0.875087
eletroportateis                  54  1.518519    1.0  0.770708

👇 Create a `worst_products_review` DataFrame which contains the `worst_products` only.

In [ ]:
worst_products_reviews = df[df.product_category_name.isin(worst_products.index)]
worst_products_reviews[["review_id", 
                        "review_score", 
                        "product_category_name",
                        "full_review_cleaned",
                        "most_important_topic",
                        "most_important_words"]
      ]

,review_id,review_score,product_category_name,full_review_cleaned,most_important_topic,most_important_words
112,ff722b4c68783a4459a3adb9bb4e1d0d,3,informatica_acessorios,produto chegou pc nao conseguiu reconhecer por...,0,"[recebi, nao, comprei, nao recebi, so]"
169,5f938e5f5f2e9a75710b54feeb9ea610,1,eletrodomesticos,medio peca nao serviu,2,"[bom, entrega, produto, prazo, nao]"
212,6b341682ab39af9fa00d72d7388c903b,3,automotivo,nota comprar produto correto capa interno ser ...,1,"[produto, nao, veio, qualidade, defeito]"
253,b736ff4204044e49e39062584d06fa74,1,eletroportateis,loja anuncia produto entrega outro,2,"[bom, entrega, produto, prazo, nao]"
272,6f885e2fd69412264d9e74e805175d53,1,informatica_acessorios,recebi bem rapido porem veio outro produto nao...,1,"[produto, nao, veio, qualidade, defeito]"
...,...,...,...,...,...,...
98392,e9f0bde9a98ff79305964e9033ed8cd2,1,informatica_acessorios,produto chegou faltando pessimas condicoes,0,"[recebi, nao, comprei, nao recebi, so]"
98401,44380245bf1a49875542ea0e7f216986,1,informatica_acessorios,nao recebi produto aind assim receber produto ...,0,"[recebi, nao, comprei, nao recebi, so]"
98512,970ae7c9df61910a0a30c38670db7ae5,3,informatica_acessorios,probleminha pasta multiuso pra notebook pedi c...,0,"[recebi, nao, comprei, nao recebi, so]"
98527,504403288b387de1c95a193909ba5dfd,3,perfumaria,chegou bem rapidinho produto razoavel,2,"[bom, entrega, produto, prazo, nao]"


### (4.3). Topics for the worst products

❓ What are the topics for the worst products ❓

In [ ]:
worst_products_reviews["most_important_topic"].value_counts()

1    590
0    579
2    371
Name: most_important_topic, dtype: int64

In [ ]:
bad_frequency = list(worst_products_reviews["most_important_topic"].value_counts().index)
bad_frequency

[1, 0, 2]

In [ ]:
[topic_word_mixture[i] for i in bad_frequency]

[['produto', 'nao', 'veio', 'qualidade', 'defeito'],
 ['recebi', 'nao', 'comprei', 'nao recebi', 'so'],
 ['bom', 'entrega', 'produto', 'prazo', 'nao']]

## (5) 🎁 Sellers...

* What kind of products were sold by the worst sellers ?
* What are the main reviews for the worst sellers ?

### (5.1) Worst Sellers

In [ ]:
sellers = Seller().get_training_data()
sellers.columns

Index(['seller_id', 'seller_city', 'seller_state', 'delay_to_carrier',
       'wait_time', 'date_first_sale', 'date_last_sale', 'months_on_olist',
       'share_of_one_stars', 'share_of_five_stars', 'review_score',
       'cost_of_reviews', 'n_orders', 'quantity', 'quantity_per_order',
       'sales', 'revenues', 'profits'],
      dtype='object')

👇 Select the 10 worst sellers and store them into a variable called `worst_sellers`

In [ ]:
worst_sellers = sellers[["seller_id", "review_score", "profits"]].sort_values(
    by = "profits", 
    ascending = True).head(10)
worst_sellers

,seller_id,review_score,profits
769,6560211a19b47992c3666cc44a7e94c0,3.937093,-21519.517
2358,4a3ca9315b744ce9f8e9374361493884,3.827873,-17752.708
1357,cc419e0650a3c5ba77189a1882b7556a,4.077586,-17581.158
945,ea8482cd71df3c1969d7b9473ff13abc,3.997371,-15542.248
315,8b321bb669392f5163d04c59e235e066,4.074547,-12886.431
453,1f50f920176fa81dab994f9023523100,4.130342,-9926.079
1213,d2374cbcbb3ca4ab1086534108cc3ab7,3.751880,-9017.608
1132,7c67e1448b00f6e969d365cea6b010ab,3.488798,-8017.611
2023,cca3071e3e9bb7d12640c9fbe2301306,3.846369,-7379.011
2687,1835b56ce799e6a4dc4eddc053f04066,3.637441,-7135.579


### (5.2) Products sold by the worst sellers

In [ ]:
products = Product().get_training_data() [["product_id", "category"]]
products

,product_id,category
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery
1,6a2fb4dd53d2cdb88e0432f1284a004c,perfumery
2,0d009643171aee696f4733340bc2fdd0,perfumery
3,b1eae565a61935e0011ee7682fef9dc9,perfumery
4,8da90b37f0fb171b4877c124f965b1f6,perfumery
...,...,...
31479,13b25797ad1e6d6c7cd3cbeb35f8ee7a,furniture_mattress_and_upholstery
31480,dc36a7859b743d8610a2bbbaea26ece9,furniture_mattress_and_upholstery
31481,107fde0930956120d1e13dd1062fbb46,furniture_mattress_and_upholstery
31482,726b4e18f00255e2e63491bcba3f60b8,furniture_mattress_and_upholstery


❓ What are the types of products sold by the worst sellers ❓

In [ ]:
sellers_product_category = data["order_items"].merge(products, 
                                             on = "product_id", how = "left")[["seller_id", "category"]]

sellers_product_category

,seller_id,category
0,48436dade18ac8b2bce089ec2a041202,cool_stuff
1,dd7ddc04e1b6c2c614352b383efe2d36,pet_shop
2,5b51032eddd242adc84c38acab88f23d,furniture_decor
3,9d7a1d34a5052409006425275ba1c2b4,perfumery
4,df560393f3a51e74553ab94004ba5c87,garden_tools
...,...,...
112645,b8bc237ba3788b23da09c0f1f3a3288c,housewares
112646,f3c38ab652836d21de61fb8314b69182,computers_accessories
112647,c3cfdc648177fdbbbb35635a37472c53,sports_leisure
112648,2b3e4a2a3ea8e01938cabda2a3e5cc79,computers_accessories


In [ ]:
sellers_product_category.groupby("seller_id").count()

,category
seller_id,
0015a82c2db000af6aaaf3ae2ecb0532,3
001cca7ae9ae17fb1caed9dfb1094831,239
001e6ad469a905060d959994f1b41e4f,0
002100f778ceb8431b7a1020ff7ab48f,55
003554e2dce176b5555353e4f3555ac8,0
...,...
ffcfefa19b08742c5d315f2791395ee5,0
ffdd9f82b9a447f6f8d4b91554cc7dd3,20
ffeee66ac5d5a62fe688b9d26f83f534,14


### (5.3) Categories and topics for the worst sellers

🎁 Here are some useful functions:
- `focus_seller(seller_id)` to show the product categories sold by a seller
- `bad_reviews_seller` to show to topwords of the most frequent topic for one seller

In [ ]:
def focus_seller(seller_id):
    return sellers_product_category[sellers_product_category.seller_id == seller_id].value_counts()

In [ ]:
bad_reviews_sellers = df.merge(data["order_items"])
bad_reviews_sellers.head(3)

,review_id,length_review,review_score,order_id,product_category_name,full_review,full_review_cleaned,most_important_topic,most_important_words,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,e233e51d11511bf30e568c76360ace52,97,1,548df2c6e5f089574614894bca78acf5,eletronicos,recebi somente 1 controle Midea Split ESTILO....,recebi somente controle midea split estilo fal...,0,"[recebi, nao, comprei, nao recebi, so]",1,62ad9a8972411e333e16347051a98e2a,128639473a139ac0f3e5f5ade55873a5,2018-01-16 16:49:36,28.90,3.94
1,e233e51d11511bf30e568c76360ace52,97,1,548df2c6e5f089574614894bca78acf5,eletronicos,recebi somente 1 controle Midea Split ESTILO....,recebi somente controle midea split estilo fal...,0,"[recebi, nao, comprei, nao recebi, so]",2,566a4f2c4385f36d15c00dfcaae132d1,8b321bb669392f5163d04c59e235e066,2018-01-16 16:49:36,21.90,26.26
2,8b230a1373c6dc4bd867099fda1d7039,60,3,071251fe3b3493294536f03737a8a679,ferramentas_jardim,Eu comprei duas unidades e só recebi uma e ag...,comprei duas unidades so recebi agora faco,0,"[recebi, nao, comprei, nao recebi, so]",1,d2085f7e0f9533605386960fc7e987ec,aac29b1b99776be73c3049939652091d,2017-11-06 02:56:07,12.49,15.10


In [ ]:
def bad_reviews_seller(bad_reviews_sellers, seller_id):
    mask = (bad_reviews_sellers.seller_id == seller_id)
    temp = bad_reviews_sellers[mask]
    most_frequent_topic_seller = list(temp.most_important_topic.value_counts().head(1).index)[0]
    return topic_word_mixture[most_frequent_topic_seller]

❓For each of these worst sellers, show the most frequent product categories and words ❓

In [ ]:
for worst_seller in worst_sellers["seller_id"]:
    print("-"*50)
    print(f"Focusing on the seller #{worst_seller}...")
    print(focus_seller(worst_seller))
    print(bad_reviews_seller(bad_reviews_sellers, worst_seller))
    

--------------------------------------------------
Focusing on the seller #6560211a19b47992c3666cc44a7e94c0...
seller_id                         category                 
6560211a19b47992c3666cc44a7e94c0  watches_gifts                1624
                                  fashion_bags_accessories      340
                                  audio                          32
                                  perfumery                      13
                                  computers_accessories          12
                                  sports_leisure                  7
                                  construction_tools_safety       1
dtype: int64
['produto', 'nao', 'veio', 'qualidade', 'defeito']
--------------------------------------------------
Focusing on the seller #4a3ca9315b744ce9f8e9374361493884...
seller_id                         category       
4a3ca9315b744ce9f8e9374361493884  bed_bath_table     1566
                                  home_confort        243
            

🏁 Congratulations. You've learned some basics of NLP (Preprocessing + Vectorizing + NB/LDA) and we combined this new "expertise" with Decision Science

💾 Don't forget to `git add / commit / push`